In [1]:
import xlrd
import math as m


class nodo():
	def __init__(self,ide,nome,lat,lon,stato,identificativo,defi,pot,coll,adia): 
		self.ide=ide
		self.lat=lat
		self.lon=lon
		self.nome=nome
		self.stato=stato
		self.identificativo=identificativo
		self.defi=defi
		self.pot=pot
		self.coll=coll
		self.adia=adia


def dist2punti(latA,lonA,latB,lonB):
	Rt=6378.388
	lat_A = (m.pi*latA)/180.
	lat_B = (m.pi*latB)/180.
	lon_A = (m.pi*lonA)/180.
	lon_B = (m.pi*lonB)/180.
	dl=abs(lon_A-lon_B)

	p = m.acos((m.sin(lat_B)*m.sin(lat_A))+(m.cos(lat_B)*m.cos(lat_A)*m.cos(dl)))
	if(lon_A>0 and lon_B>0): 
		if(lon_B > lon_A): 
			p=p
		else:
			p=float('inf')
	if(lon_A<0 and lon_B<0): 
		if(lon_B > lon_A): 
			p=p
		else:
			p=float('inf')
	if(lon_A>0 and lon_B<0):
		if((lon_A+abs(lon_B))<m.pi): 
			p=float('inf')
		else:
			p=p
	if(lon_A<0 and lon_B>0):
		if((lon_B+abs(lon_A))<m.pi): 
			p=p
		else:
			p=float('inf')
	return(p*Rt) 


dati=xlrd.open_workbook('/Users/Tonino222power/Desktop/City.xlsx')
foglio = dati.sheet_by_name('Foglio1')
nR=foglio.nrows
nC=foglio.ncols


citta_inizio="London"
inizio="1826645935"



nome=[]
lat=[]
log=[]
stato=[]
stato2=[]
pop=[]
iden=[]
pesi=[]
nodi=[]


for i in range(1,nR):	
	nome.append(foglio.cell(i,0).value)
	lat.append(foglio.cell(i,2).value)
	log.append(foglio.cell(i,3).value)
	stato.append(foglio.cell(i,6).value)
	stato2.append(foglio.cell(i,7).value)
	pop.append(foglio.cell(i,9).value)
	iden.append(str(int(foglio.cell(i,10).value)))

print("Lettura dati da file terminata")


for i in range(0,nR-1):
	if(pop[i]==''):
		pop[i]=0

print("Controllo popolazioni terminato")



elim=[]
elim1=[]

for i in range(0,nR-1):
	for j in range(0,nR-1):
		if(lat[j]==lat[i] and log[j]==log[i] and iden[j]!=iden[i]):
			print(j)
			elim.append(j)

for i in range(0,nR-1):
	for j in range(0,nR-1):
		if(iden[j]==iden[i] and lat[j]!=lat[i] and log[j]!=log[i]):
			print(k)
			elim1.appen(j)

for j in elim:
	print("rimosso per coordinate coincidenti: ",iden[j])
	nome.pop(j)
	lat.pop(j)
	log.pop(j)
	stato.pop(j)
	stato2.pop(j)
	pop.pop(j)
	iden.pop(j)

for j in elim1:
	print("rimosso per identificativo coincidente: ",iden[j])
	nome.pop(j)
	lat.pop(j)
	log.pop(j)
	stato.pop(j)
	stato2.pop(j)
	pop.pop(j)
	iden.pop(j)


print(len(nome))
nR=len(nome)

citta_fine=citta_inizio+"_fine"
fine="1111111111"

for i in range(0,nR):
	if(iden[i]==inizio):
		nome.append(citta_fine)
		lat.append(lat[i])
		log.append(log[i]-0.1)
		stato.append(stato[i])
		stato2.append(stato2[i])
		pop.append(pop[i])
		iden.append(fine)


print("Numerosità città")
print(len(nome))
nN=len(nome)


for i in range(0,nN):
	vicini=[]
	for j in range(0,nN):
		if(iden[i]==inizio and iden[j] == fine):
			continue
		if(iden[j]==inizio):
			continue
		if(iden[i]!=iden[j]):
			d=dist2punti(lat[i],log[i],lat[j],log[j])
			if(len(vicini)==0):
				vicini.append([iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])
			else:	
				for k in range(0,len(vicini)):
					if(d<vicini[k][1]):
						vicini.insert(k,[iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])
						if(len(vicini)>3):					
							vicini.pop(3)
						break
					if(k==2 and len(vicini)<3):
						vicini.append([iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])

	diz={}
	for l in range(0,len(vicini)):

		pes=2**(l+1)
		if(vicini[l][2]>200000):
			pes=pes+2

		if(vicini[l][3] != vicini[l][4]):
			pes=pes+2
		diz[vicini[l][0]]=pes 

	print("Ancora "+str(nN-i)+" citta' da controllare" )

	pesi.append(diz)

	nodi.append(nodo(i,nome[i],lat[i],log[i],stato[i]+"-"+stato2[i],iden[i],True,float('inf'),"",pesi[i])) 

print("Algoritmo di calcolo del grafo utilizzando l'algoritmo Dijkstra")



for x in nodi:
	if(x.identificativo == inizio):
		x.defi=False
		x.pot=0
		x.coll=inizio
		i=x.ide
k=0
while True:
	for x in nodi[i].adia.keys():
		for y in nodi:
			if (y.identificativo == x and y.defi ):
				somma=nodi[i].pot + nodi[i].adia[x]
				if(y.pot>somma):
					y.pot=somma
					y.coll=nodi[i].identificativo
	m=float('inf')
	for z in nodi:
		if(z.defi and z.pot<m):
			m=z.pot
			n=z.ide
	
	nodi[n].defi=False
	i=n
	print(nodi[i].nome, i)

	if(nodi[i].identificativo == fine ):
		lung=nodi[i].pot
		break
k=k+1

percorso=[]
percorso_nomi=[]


s=fine
while True:
	percorso.append(s)
	for x in nodi:
		if(x.identificativo == s):
			s=x.coll
	if(s==inizio):
		percorso.append(s)
		break


for x in percorso:
	for y in nodi:
		if(y.identificativo==x):
			percorso_nomi.append(y.nome+" ("+y.stato+")")

strada=""
for i in range(0,len(percorso_nomi)-1):
	strada=strada+percorso_nomi[len(percorso_nomi)-1-i]+" - "
strada=strada+" "+percorso_nomi[0]

print("-------------------Risultato----------------------------")
print("Percorso terminato in "+str(lung)+" ore o "+str(float(lung)/24.)+" giorni. (tempo minimo)" )
print("Tragitto: " + strada)
print("--------------------------------------------------------")

Lettura dati da file terminata
Controllo popolazioni terminato
3913
2942
3976
19829
1155
5153
183
1426
6543
10176
3837
5609
10784
10848
5232
11750
4146
22963
4951
5365
5400
5744
18159
18158
1962
6709
rimosso per coordinate coincidenti:  1484228590
rimosso per coordinate coincidenti:  1484980825
rimosso per coordinate coincidenti:  1418863956
rimosso per coordinate coincidenti:  1840142185
rimosso per coordinate coincidenti:  1484504936
rimosso per coordinate coincidenti:  1840000395
rimosso per coordinate coincidenti:  1484559591
rimosso per coordinate coincidenti:  1076190259
rimosso per coordinate coincidenti:  1516928100
rimosso per coordinate coincidenti:  1840016003
rimosso per coordinate coincidenti:  1032425368
rimosso per coordinate coincidenti:  1840018841
rimosso per coordinate coincidenti:  1356820046
rimosso per coordinate coincidenti:  1076277699
rimosso per coordinate coincidenti:  1724017694
rimosso per coordinate coincidenti:  1804965832
rimosso per coordinate coinciden

Ancora 26348 citta' da controllare
Ancora 26347 citta' da controllare
Ancora 26346 citta' da controllare
Ancora 26345 citta' da controllare
Ancora 26344 citta' da controllare
Ancora 26343 citta' da controllare
Ancora 26342 citta' da controllare
Ancora 26341 citta' da controllare
Ancora 26340 citta' da controllare
Ancora 26339 citta' da controllare
Ancora 26338 citta' da controllare
Ancora 26337 citta' da controllare
Ancora 26336 citta' da controllare
Ancora 26335 citta' da controllare
Ancora 26334 citta' da controllare
Ancora 26333 citta' da controllare
Ancora 26332 citta' da controllare
Ancora 26331 citta' da controllare
Ancora 26330 citta' da controllare
Ancora 26329 citta' da controllare
Ancora 26328 citta' da controllare
Ancora 26327 citta' da controllare
Ancora 26326 citta' da controllare
Ancora 26325 citta' da controllare
Ancora 26324 citta' da controllare
Ancora 26323 citta' da controllare
Ancora 26322 citta' da controllare
Ancora 26321 citta' da controllare
Ancora 26320 citta' 

Ancora 26112 citta' da controllare
Ancora 26111 citta' da controllare
Ancora 26110 citta' da controllare
Ancora 26109 citta' da controllare
Ancora 26108 citta' da controllare
Ancora 26107 citta' da controllare
Ancora 26106 citta' da controllare
Ancora 26105 citta' da controllare
Ancora 26104 citta' da controllare
Ancora 26103 citta' da controllare
Ancora 26102 citta' da controllare
Ancora 26101 citta' da controllare
Ancora 26100 citta' da controllare
Ancora 26099 citta' da controllare
Ancora 26098 citta' da controllare
Ancora 26097 citta' da controllare
Ancora 26096 citta' da controllare
Ancora 26095 citta' da controllare
Ancora 26094 citta' da controllare
Ancora 26093 citta' da controllare
Ancora 26092 citta' da controllare
Ancora 26091 citta' da controllare
Ancora 26090 citta' da controllare
Ancora 26089 citta' da controllare
Ancora 26088 citta' da controllare
Ancora 26087 citta' da controllare
Ancora 26086 citta' da controllare
Ancora 26085 citta' da controllare
Ancora 26084 citta' 

Ancora 25876 citta' da controllare
Ancora 25875 citta' da controllare
Ancora 25874 citta' da controllare
Ancora 25873 citta' da controllare
Ancora 25872 citta' da controllare
Ancora 25871 citta' da controllare
Ancora 25870 citta' da controllare
Ancora 25869 citta' da controllare
Ancora 25868 citta' da controllare
Ancora 25867 citta' da controllare
Ancora 25866 citta' da controllare
Ancora 25865 citta' da controllare
Ancora 25864 citta' da controllare
Ancora 25863 citta' da controllare
Ancora 25862 citta' da controllare
Ancora 25861 citta' da controllare
Ancora 25860 citta' da controllare
Ancora 25859 citta' da controllare
Ancora 25858 citta' da controllare
Ancora 25857 citta' da controllare
Ancora 25856 citta' da controllare
Ancora 25855 citta' da controllare
Ancora 25854 citta' da controllare
Ancora 25853 citta' da controllare
Ancora 25852 citta' da controllare
Ancora 25851 citta' da controllare
Ancora 25850 citta' da controllare
Ancora 25849 citta' da controllare
Ancora 25848 citta' 

Ancora 25641 citta' da controllare
Ancora 25640 citta' da controllare
Ancora 25639 citta' da controllare
Ancora 25638 citta' da controllare
Ancora 25637 citta' da controllare
Ancora 25636 citta' da controllare
Ancora 25635 citta' da controllare
Ancora 25634 citta' da controllare
Ancora 25633 citta' da controllare
Ancora 25632 citta' da controllare
Ancora 25631 citta' da controllare
Ancora 25630 citta' da controllare
Ancora 25629 citta' da controllare
Ancora 25628 citta' da controllare
Ancora 25627 citta' da controllare
Ancora 25626 citta' da controllare
Ancora 25625 citta' da controllare
Ancora 25624 citta' da controllare
Ancora 25623 citta' da controllare
Ancora 25622 citta' da controllare
Ancora 25621 citta' da controllare
Ancora 25620 citta' da controllare
Ancora 25619 citta' da controllare
Ancora 25618 citta' da controllare
Ancora 25617 citta' da controllare
Ancora 25616 citta' da controllare
Ancora 25615 citta' da controllare
Ancora 25614 citta' da controllare
Ancora 25613 citta' 

...

Koforidua 3794
San Andrés 4956
La Paz 5847
La Unión 6560
Somoto 6977
Damongo 9382
Cancún 1009
Cape Coast 3476
Estelí 3760
Puerto Limón 5218
Fatick 6737
Isla Mujeres 7400
Nacaome 7522
Greytown 26030
Tamale 1449
San Miguel 2543
Tema 3149
Jinotega 3646
Juigalpa 5255
Winneba 11533
San Lorenzo 11684
Yendi 11696
Sixaola 20140
Accra 340
Colón 2324
Pinar del Río 2836
Kaolack 3041
Diourbel 4562
Gaigirgordub 9324
La Cruz 20943
Comayagüela 566
Matagalpa 3268
Ciudad Choluteca 3986
Ho 4801
Liberia 5707
San Carlos 7567
Nalerigu 9152
La Palma 12467
Bassar 14560
Guabito 21696
La Chorrera 5006
Bolgatanga 5111
La Palma 8562
Kaffrine 9131
Buabidi 9284
Aflao 10311
Sansanné-Mango 11723
Cañas 15338
El Castillo de La Concepción 15527
Almirante 21889
Tegucigalpa 628
Sokodé 4183
Kpalimé 4555
Nueva Gerona 5687
Bocas del Toro 7609
Kara 8943
Unión Chocó 9446
Bawku 10075
Balboa 10466
Hohoe 10817
Bahía Honda 11666
Farafenni 12369
San Cristóbal 13307
Chinandega 3355
Granada 4408
Natitingou 4847
Atakpamé 4852
Santiag

Kabalo 13198
Dembī Dolo 13513
Ed Damazin 2858
Kasese 5019
Ruhengeri 5236
Fort Portal 5749
Kyenjojo 8852
Kakumiro 8993
Kyegegwa 9015
Kasongo 10476
Muhanga 11213
Bukama 12104
Nyunzu 17158
Gorē 20636
Katwe 25938
Nimule 26025
Juba 759
Gedaref 1737
Nebbi 6328
Torit 7164
Bundibugyo 8501
Amuru 9280
Gitarama 9659
Manono 11464
Gīmbī 12658
Kalemie 2638
Nek’emtē 4922
Mubende 7006
Pakwach 7100
Kisoro 7521
Rubirizi 8769
Kibiito 8895
Lamwo 9066
Kambove 12302
Ruhango 16207
Mwenga 26346
Kolwezi 1573
Jīma 2920
Kigoma 3725
Kitgum 5307
Hāgere Hiywet 5567
Kiboga 7374
Rubanda 8877
Kasanda 8931
Kamwenge 8940
Nwoya 9355
Kapoeta 22981
Teseney 25868
Bukavu 1487
Likasi 1550
Gulu 3387
Giyon 4032
Kipushi 4170
Kabale 5725
Bushenyi 8800
Bulisa 9158
Barentu 9308
Kasulu 12693
Moba 20318
Luanza 26407
Kigali 730
Kassala 1288
Mityana 5772
Kirundo 7857
Kaabong 8461
Cibitoke 8751
Ibanda 8833
Nsiika 8893
Ntara 8901
Mparo 8909
Sembabule 8990
Palenga 9088
Pader 9113
Kyankwanzi 9326
Hosa’ina 9650
Ak’ordat 21254
Kanyato 26445


Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitt

Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146
Pathanāmthitta 12146


KeyboardInterrupt: 